# Encryption Composers

> Located in [./lib](./lib/).

In this project, composers (loose shorthand reference), are a unique classes to this project that enable rapidly tested encryption algorithms whilst collecting useful metrics. Overtime, various forms of composers will be created. In essence, composers are high-level encryption-algorithm harnesses.


# Simple Encryption Composer

Built for a single algorithm, this harness aims to make simple usages of new algorithms to compare them to others, and test for mock-production processes.

## Fire & Forget Ideology

The idea is to fire the algorithm in the composer and forget about about logistics while you review the results.


In [1]:
from lib.notebook import (
    algorithm,
    with_key,
    generate_key,
    AlgorithmContext,
    ComposerSession,
    quick_test,
)

# -----------------------------------------------------------------------------
# Prototype Algorithm Development
# -----------------------------------------------------------------------------
# Use decorators for logistics (key injection, context, metrics).
# Write your own experimental crypto logic in the methods.

KEY = generate_key(32)


@algorithm("XOR-Prototype")
@with_key(KEY)
class XORPrototype:
    """
    Simple XOR-based prototype for testing the framework.
    NOT cryptographically secure - just a development placeholder.
    """

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Prototype: simple repeating-key XOR
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # XOR is symmetric
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))


# Verify the prototype works
quick_test(XORPrototype())

Testing: XOR-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 0.028ms>
Decrypt: <AlgorithmResult: 18 bytes, 0.007ms>
✓ Round-trip successful!


In [2]:
# -----------------------------------------------------------------------------
# Composer Session - Benchmark Prototype vs Production Algorithms
# -----------------------------------------------------------------------------
# Register your prototype alongside proven algorithms from lib/algorithms
# to compare performance and validate behavior.

from lib.algorithms import create_aes256gcm

# Create session and register algorithms
session = ComposerSession()

# Register our prototype
session.register(XORPrototype())

# Register proven algorithm from lib/algorithms (wrapped for notebook API)
# TODO: Add notebook-compatible wrappers for lib/algorithms

print("Registered algorithms:", session.list_algorithms())

# Test all registered algorithms
print("\n=== Round-trip Tests ===")
test_results = session.test_all()
for name, passed in test_results.items():
    status = "✓ PASS" if passed else "✗ FAIL"
    print(f"  {name}: {status}")

Registered algorithms: ['XOR-Prototype']

=== Round-trip Tests ===
  XOR-Prototype: ✓ PASS


In [3]:
# -----------------------------------------------------------------------------
# Benchmark Prototype Performance
# -----------------------------------------------------------------------------

print("=== Prototype Benchmark (10KB, 50 iterations) ===\n")
comparison = session.compare(data_size=10_000, iterations=50)

print(f"{'Algorithm':<20} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput (MB/s)':<15}")
print("-" * 65)
for entry in comparison:
    print(f"{entry['algorithm']:<20} {entry['avg_encrypt_ms']:<15} {entry['avg_decrypt_ms']:<15} {entry['throughput_mbps']:<15}")

=== Prototype Benchmark (10KB, 50 iterations) ===

Algorithm            Encrypt (ms)    Decrypt (ms)    Throughput (MB/s)
-----------------------------------------------------------------
XOR-Prototype        0.964           0.973           10.38          


In [4]:
# -----------------------------------------------------------------------------
# Detailed Benchmarks - Scaling Analysis
# -----------------------------------------------------------------------------

print("=== Scaling Analysis ===\n")
benchmarks = session.benchmark_all(
    data_sizes=[100, 1_000, 10_000, 100_000],
    iterations=20
)

for algo_name, results in benchmarks.items():
    print(f"{algo_name}:")
    print(f"  {'Size':<12} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput':<15}")
    print(f"  {'-'*55}")
    for bench in results["benchmarks"]:
        size_label = f"{bench['size_bytes']:,} B"
        print(f"  {size_label:<12} {bench['avg_encrypt_ms']:<15} {bench['avg_decrypt_ms']:<15} {bench['throughput_mbps']:<15} MB/s")

=== Scaling Analysis ===

XOR-Prototype:
  Size         Encrypt (ms)    Decrypt (ms)    Throughput     
  -------------------------------------------------------
  100 B        0.022           0.012           4.6             MB/s
  1,000 B      0.115           0.112           8.72            MB/s
  10,000 B     1.106           1.13            9.04            MB/s
  100,000 B    9.844           9.675           10.16           MB/s


In [5]:
# -----------------------------------------------------------------------------
# Session Metrics - Aggregated Statistics
# -----------------------------------------------------------------------------

print("=== Session Report ===\n")
report = session.report()

for algo_name, metrics in report.items():
    print(f"{algo_name}:")
    print(f"  Operations: {metrics['encrypt_calls']} encrypt, {metrics['decrypt_calls']} decrypt")
    print(f"  Avg timing: {metrics['avg_encrypt_ms']:.3f}ms encrypt, {metrics['avg_decrypt_ms']:.3f}ms decrypt")
    print(f"  Total data: {metrics['total_bytes_processed']:,} bytes")
    if metrics['errors'] > 0:
        print(f"  Errors: {metrics['errors']}")
    print()

=== Session Report ===

XOR-Prototype:
  Operations: 1 encrypt, 1 decrypt
  Avg timing: 0.015ms encrypt, 0.005ms decrypt
  Total data: 36 bytes

